In [ ]:
# RTN Time Domain
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import seaborn as sns
from scipy.signal import find_peaks
from scipy.stats import expon
from scipy.stats import poisson
import pywt
from scipy.optimize import curve_fit

In [ ]:
def plot_raw(df, y_column="Id", name = "plot", color="blue"):
    # Create a line plot trace
    trace = go.Scatter(x=df['Time'], y=df[y_column],
                       mode='lines',
                       name=name,
                       line=dict(color=color))

    # Create a layout
    layout = go.Layout(title=name,
                       xaxis=dict(title='Time'),
                       yaxis=dict(title='Current'))

    # Create a Figure and add the trace
    fig = go.Figure(data=[trace], layout=layout)

    return fig

In [ ]:
def notch_filter(df, samp_freq):
    notch_freq = [60 * i for i in range(1, int(samp_freq / 120))]
    y_notched = df.copy()
    y_notched = y_notched['Id']
    quality_factor = 10
    for filter_f in tqdm(notch_freq):
        b_notch, a_notch = signal.iirnotch(filter_f, quality_factor, samp_freq)
        freq, h = signal.freqz(b_notch, a_notch, fs = samp_freq)

        y_notched = signal.filtfilt(b_notch, a_notch, y_notched)

    plt.subplot(211)
    plt.plot(df['Time'],df['Id'], color = 'r')
    plt.subplot(212)
    plt.plot(df['Time'], y_notched, color = 'r')

    df_denoised = pd.DataFrame()
    df_denoised['Time'] = df['Time']
    df_denoised['Id_original'] = df['Id']
    df_denoised['Id'] = y_notched

    return df_denoised

In [ ]:
def Perform_KDE(df, column="Id", n_peaks=4, type="normal"):
    data = df[column]

    # Perform KDE once and get data
    sns.kdeplot(data, fill=True)
    KDE_model = sns.kdeplot(data).get_lines()[0]
    print("KDE_model:", KDE_model)
    x = KDE_model.get_xdata()
    y = KDE_model.get_ydata()
    plt.close()  # Close the initial plot

    # Debug prints
    print("len(y)=", len(y))
    print("len(x)=", len(x))

    # Find peaks in the KDE distribution
    peaks, _ = find_peaks(y)

    # Find local minima by inverting the KDE y-data and finding peaks
    inverted_y = -y
    local_minima, _ = find_peaks(inverted_y)
    local_minima = np.array([int(x) for x in local_minima])
    minima = local_minima[np.argsort(inverted_y[local_minima])[-len(local_minima):]]

    # Get the top peaks based on height
    top_peaks = peaks[np.argsort(y[peaks])[-n_peaks:]]

    print("minimais: ", minima)
    print("peaks: ", top_peaks)

    # Plot again with marked points
    sns.kdeplot(data, fill=True)
    plt.scatter(x[top_peaks], y[top_peaks], color='red', s=30, marker='o')
    plt.scatter(x[local_minima], y[local_minima], color='green', s=30, marker='o')

    if type == "log":
        plt.gca().set_yscale("log")
    plt.show()

    # Retrieve the x-values corresponding to these peaks
    peak_values = x[top_peaks]
    min_values_rank_by_density = x[minima]

    # Safeguard against potential IndexError
    valid_minima = local_minima[local_minima < len(x)]
    min_values = x[valid_minima]

    peak_values_rank_by_density = peak_values.copy()
    peak_values = np.sort(peak_values)

    return peak_values, peak_values_rank_by_density, min_values, min_values_rank_by_density


In [ ]:
def wavelet_transform(df, threshold=1e-5, level_val=7):
    df_denoise1 = df.copy()
    noisy_signal = df_denoise1["Id"]
    t = df_denoise1["Time"]

    coeffs = pywt.wavedec(noisy_signal, 'haar', level=7)

    # Set a threshold value and apply thresholding to the detail coefficients
    coeffs_thresholded = [pywt.threshold(c, threshold, mode='hard') for c in coeffs]
    # Reconstruct the signal from the thresholded coefficients
    denoised_signal = pywt.waverec(coeffs_thresholded, 'haar')
    denoised_signal = denoised_signal# [1:]

    plt.subplot(3, 1, 2)
    plt.plot(t, noisy_signal, label='Noisy Signal')
    plt.title("Signal with Noise")
    plt.legend()

    plt.subplot(3, 1, 3)
    if len(t) != len(denoised_signal):
        # plt.plot(t, denoised_signal[:-1], label='Denoised Signal')
        denoised_signal = denoised_signal[:-1]
    plt.plot(t, denoised_signal, label='Denoised Signal')
    plt.title("Denoised Signal")
    plt.legend()

    plt.tight_layout()
    plt.show()
    df_denoise1["Id_denoised"] = denoised_signal

    plot_fig = plot_raw(df_denoise1, y_column="Id_denoised")
    plot_fig.show()
    return df_denoise1

In [ ]:
def record_time(df, cutoff, threshold, column):
    emission_time = []
    capture_time = []
    emission_start_time = None
    emission_end_time = None
    capture_start_time = None
    capture_end_time = None
    # iterate over dataframe rows
    for idx, row in df.iterrows():
        if row[column] < threshold and emission_start_time is None:
            emission_start_time = row['Time']

        elif row[column] > threshold and capture_start_time is None:
            capture_start_time = row['Time']

        elif row[column] > threshold and emission_start_time is not None:
            emission_end_time = row['Time']
            duration = emission_end_time - emission_start_time
            if (duration >= cutoff):
                emission_time.append((emission_start_time, emission_end_time, duration))
                emission_start_time = None

        elif row[column] < threshold and capture_start_time is not None:
            capture_end_time = row['Time']
            duration = capture_end_time - capture_start_time
            # print(duration)
            if (duration >= cutoff):
                capture_time.append((capture_start_time, capture_end_time, duration))
                capture_start_time = None

    # When we reach the end of the data, but the capture/emission even has not been changed.
    if emission_start_time is not None:
        emission_end_time = df.loc[idx, 'Time']
        duration = emission_end_time - emission_start_time
        if (duration >= cutoff):
                emission_time.append((emission_start_time, emission_end_time, duration))
                emission_start_time = None

    elif capture_start_time is not None:
        end_time = df.loc[idx, 'Time']
        duration = capture_end_time - capture_start_time
        if (duration >= cutoff):
                capture_time.append((capture_start_time, capture_end_time, duration))
                start_time = None

    return emission_time, capture_time

In [ ]:
def emission_capture_ratio(df, cutoff = 0.02, change_threshold = False, threshold_provide = 0, y_column="Id_denoised"):
    fig = plot_raw(df, y_column)
    fig.show()

    if change_threshold:
        threshold = threshold_provide
    else:
        peak_values, peak_values_rank_by_density, min_values, min_values_rank_by_density = Perform_KDE(df, column="Id_denoised") #change
        threshold =  (peak_values_rank_by_density[-1] + peak_values_rank_by_density[-2]) / 2
        print("threshold is: ", threshold)

    emission, capture = record_time(df, cutoff, threshold, y_column) #change
    # capture1 = [x[2] for x in capture]
    df_emission_total = sum(data[2] for data in emission)
    df_capture_total = sum(data[2] for data in capture)
    emission_capture =  df_emission_total / (df_emission_total + df_capture_total)
    return emission_capture, emission, capture

In [ ]:
# RFSOI
def plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.6_Vd0.3_10kHz_2000000_2K_1MA_0.csv" ):
    df2K_6= pd.read_csv(addr)
    df2K_6.columns = ["Time", "Id"]
    # Remember to change the sampling frequency!
    df_denoise2K_6 = notch_filter(df2K_6, 10000)
    fig = plot_raw(df_denoise2K_6)
    fig.show()
    return df_denoise2K_6

In [ ]:
# 6 measurements in one run
def plot_many(names = [0.45,0.5,0.55,0.6,0.65,0.7], addr = r"C:\Users\kongx\OneDrive\Desktop\cena_2\nthin56_Vg(0.45,0.5,0.55,0.6,0.65,0.7)_Vd0.2_50kHz_2K_3900000_1MA_3.csv", sf = 10000):
    df50K_2= pd.read_csv(addr)
    df50K_2.columns = ["Time", "Id"]
    # Remember to change the sampling frequency!
    df_denoise50K_2 = notch_filter(df50K_2, sf)
    # names = [0.4,0.5,0.6,0.7,0.8,0.9]
    for i in range(6):
        fig = plot_raw(df_denoise50K_2.iloc[650000*i+2000:650000*(i+1) - 2000], name = str(names[i]))
        fig.show()
    return df_denoise50K_2

In [ ]:
def plot_RFSOI(addr=r"C:\wgfmu_example\RFSOI_pbits\nthin56_Vd(0.050,0.075,0.100,0.125,0.150,0.175)_Vg0.850_10kHz_2K_3900000_1MA.csv", fs=10000):
    df50K_5= pd.read_csv(addr)
    df50K_5.columns = ["Time", "Id"]
    names = addr[addr.find("(")+1:addr.find(")")-1].split(",")
    # Remember to change the sampling frequency!
    df_denoise50K_5 = notch_filter(df50K_5, fs)
    proccessed_data = []
    for i in range(6):
        fig = plot_raw(df_denoise50K_5.iloc[650000*i+2000:650000*(i+1) - 2000], name = str(names[i]))
        fig.show()
        proccessed_data.append(df_denoise50K_5.iloc[650000*i+2000:650000*(i+1) - 2000])

    return proccessed_data

In [ ]:
def time_to_point(df, time):
    total_time = 65
    return (int(time - df.iloc[0]['Time']) / total_time) * 650000

In [ ]:
def get_probability(df, input_start, step):
    input = []
    for i in range(0,6):
        input.append(i*step)
    points = []
    probabilities = []
    for data in df:
        point2 = wavelet_transform(data, threshold=5e-4)
        points.append(point2)
        probability, _, _ = emission_capture_ratio(point2, cutoff = 0.02)
        probabilities.append(probability)
    return input, probabilities


In [ ]:
# Step1: Get the data
df = plot_RFSOI(r"C:\wgfmu_example\RFSOI_pbits\nthin56_Vd(0.05,0.075,0.1,0.125,0.15,0.175)_Vg0.65_10kHz_2K_3900000_1MA.csv")
point15 = wavelet_transform(df[4], threshold=5e-4)

In [ ]:
emission_capture_15, _, _ = emission_capture_ratio(point15, cutoff = 0.02, change_threshold = True, threshold_provide = 144.3e-6)

In [ ]:

group2, probability_group2 = get_probability(df)

In [ ]:
# Plotting 45SPCLO Changing Vg, 2K
# This gives an example of plotting data along with the sigmoid function.
datain1 =  [0.6, 0.65,0.7]
datain2 =  [0.5,0.6,0.65,0.7,0.75]
datain3 =  [0.6,0.65, 0.7, 0.75, 0.8]
dataout1 = [0.9935712571884127, 0.8614155122869538,0.9439158799412982]
dataout2 = [0.03547703269254607,0.8064629617965031, 0.9439158799412982,0.9264051922799923,0.9264051922799923]
dataout3= [0.053,0.20856073758195928, 0.40431752649595687, 0.5536989154853669,0.5845002428167911]

def sigmoid_func(x, a, b):
    return 1 / (1 + np.exp(-a * x + b))

# Fit the sigmoid function to the data
popt1, pcov1 = curve_fit(sigmoid_func, datain1, dataout1)
x_fit1 = np.linspace(0, 1, 100)
y_fit1 = sigmoid_func(x_fit1, *popt1)

popt2, pcov2 = curve_fit(sigmoid_func, datain2, dataout2)
x_fit2 = np.linspace(0, 1, 100)
y_fit2 = sigmoid_func(x_fit2, *popt2)

popt3, pcov3 = curve_fit(sigmoid_func, datain3, dataout3)
x_fit3 = np.linspace(0, 1, 100)
y_fit3 = sigmoid_func(x_fit3, *popt3)

#
fig, ax1 = plt.subplots(figsize=(650/100, 400/100))

# Plot the first line with ax1 for the left y-axis using square markers
ax1.set_xlabel('Vg(V)')
ax1.set_ylabel('Probability')
# 77, 121, 255

# ax1.plot(x_fit1, y_fit1, color='red', alpha=0.3, linewidth=12)
# ax1.plot(x_fit1, y_fit1, color=(255/255, 100/255, 130/255), alpha=0.3, linewidth=12)
ax1.plot(x_fit2, y_fit2, color=(255/255, 153/255, 51/255), alpha=0.3, linewidth=12)
ax1.plot(x_fit3, y_fit3, color=(255/255, 51/255, 204/255), alpha=0.3, linewidth=12)
# ax1.plot(CLO_Vd_point2_Vg, CLO_Vd_point2_I, color='red', marker='o', label='Device 1: Vd=0.2', linewidth=0, markersize=10)  # Added square markers here
# ax1.plot(CLO_Vd_point15_Vg, CLO_Vd_point15_I, color='red', marker='s', label='#1: Vd=0.15', linewidth=2, markersize=10)  # Added square markers here
ax1.plot(datain1, dataout1, color=(255/255, 100/255, 130/255), marker='^', label='Vd=0.4', linewidth=0, markersize=8)  # Added square markers here
ax1.plot(datain2, dataout2, color=(255/255, 153/255, 51/255), marker='s', label='Vd=0.3', linewidth=0, markersize=8)  # Added square markers here
ax1.plot(datain3, dataout3, color=(255/255, 51/255, 204/255), marker='o', label='Vd=0.2', linewidth=0, markersize=8)


# ax1.plot(rf_temp_mag, rf_mag_percent, color='blue', marker='o', linestyle='-', label='45RFSOI', linewidth=2, markersize=10)  # Added point markers here
ax1.set_xlim(0, 1)  # Set the limit for left y-axis as percentage

# Title and layout
plt.title('Impact of Gate Voltage on the transition probability @2K')
fig.tight_layout()  # To ensure nothing is clipped

# Show plot with a legend
ax1.legend(loc='best', bbox_to_anchor=(1, 1))

# Set box and grid properties
plt.legend(prop={'weight': 'normal'})  # Set legend text weight to 'normal' (non-bold)
plt.box(True)  # Show box around plot
plt.rcParams['axes.linewidth'] = 2  # Set box linewidth to 2
# plt.rcParams['savefig.dpi']=300
plt.grid(True, linewidth=2, linestyle='--', color='black', alpha=0.25)  # Set grid properties

plt.show()


In [ ]:
df_denoise2K_3 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.3_Vd0.3_10kHz_1000000_2K_100UA_0.csv")
df_denoise2K_4 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.4_Vd0.3_10kHz_2000000_2K_100UA_0.csv")
df_denoise2K_5 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.5_Vd0.3_10kHz_2000000_2K_100UA_0.csv")
df_denoise2K_6 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.6_Vd0.3_10kHz_2000000_2K_1MA_0.csv")
emission_capture_6 = emission_capture_ratio(Vg_point6, cutoff = 0.02, change_threshold = True, threshold_provide = 130e-6, y_column="Id_denoised")
df_denoise2K_65 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.65_Vd0.3_10kHz_2000000_2K_1MA_0.csv")
emission_capture_65 = emission_capture_ratio(df_denoise2K_65, cutoff = 0.02, change_threshold = True, threshold_provide = 203.7e-6, y_column="Id")
df_denoise2K_7 = plot_vg_vd(addr=r"C:\wgfmu_example\8.3_new_start\nthin_Vg0.7_Vd0.3_10kHz_2000000_2K_1MA_0.csv")
emission_capture_7 = emission_capture_ratio(df_denoise2K_7, cutoff = 0.02, change_threshold = True, threshold_provide = 287.5e-6, y_column="Id")


In [ ]:
# For SPCLO Vd=0.2 2K
CLO_df_denoise50K_2 = plot_many(names = [0.45,0.5,0.55,0.6,0.65,0.7], addr = r"C:\Users\kongx\OneDrive\Desktop\cena_2\nthin56_Vg(0.45,0.5,0.55,0.6,0.65,0.7)_Vd0.2_50kHz_2K_3900000_1MA_3.csv", sf=50000)   # C:\Users\kongx\OneDrive\Desktop\cena_2
emission_capture_45 = 0.00133
emission_capture_5 = emission_capture_ratio(CLO_df_denoise50K_2.iloc[650000*1+2000:int(650000*(1+1)) - 2000], cutoff = 0.02, change_threshold = True, threshold_provide = 26.17e-6, y_column="Id")# CLO_df_denoise50K_2
emission_capture_55 = emission_capture_ratio(CLO_df_denoise50K_2.iloc[650000*2+2000:int(650000*(2+1)) - 2000], cutoff = 0.02, change_threshold = True, threshold_provide = 60.5e-6, y_column="Id")
emission_capture_6 = emission_capture_ratio(CLO_df_denoise50K_2.iloc[650000*3+2000:int(650000*(3+1)) - 2000], cutoff = 0.02, change_threshold = True, threshold_provide = 112.2e-6, y_column="Id")
emission_capture_65 = 0.0001
emission_capture_7 = 0.0001

In [ ]:
### Draft region ###

In [ ]:
# This is a little off-topic of this notebook. It plots time_constant v.s. temperature.

import matplotlib.pyplot as plt

# Setup for figure
font = {'family': 'times new roman', 'weight': 'normal', 'size':20}
plt.rc('font', **font)
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'  # Optional: Use amsmath package
plt.rcParams['font.family'] = 'times new roman'  # Set font family (e.g., 'serif')
# plt.rcParams['text.latex.preamble'] = r'\boldmath'  # Bold font for LaTeX-rendered text (remove this line for non-bold)

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(550/100, 600/100))

cena_temp = [100, 50, 25]
cena_capture = [0.4263, 0.7311, 3.3798]
cena_emission = [0.0157, 4.263, 6.3188]

rf_temp = [100, 85, 50, 2.5]
rf_capture = [0.004081, 0.053723, 0.204646, 0.24568]
rf_emission = [0.030442, 0.10112, 0.301484, 0.762815]
#rf_emission = [0.030442, 0.10112, 0.701484, 0.762815]

# Plot the first line with ax1 for the left y-axis using triangle-up markers
ax1.set_xlabel('Temperature [K]')
ax1.set_ylabel('Time constant [s]')

line2 = ax1.plot(cena_temp, cena_emission, color=(255/255, 102/255, 0/255), marker='o', label=r'45SPCLO $\tau_e$', linewidth=2, markersize=10)
line1 = ax1.plot(cena_temp, cena_capture, color=(255/255, 102/255, 0/255), marker='^', label=r'45SPCLO $\tau_c$', linewidth=2, markersize=10)
line4 = ax1.plot(rf_temp, rf_emission, color=(204/255, 51/255, 255/255), marker='h', label=r'45RFSOI $\tau_e$', linewidth=2, markersize=10)
line3 = ax1.plot(rf_temp, rf_capture, color=(204/255, 51/255, 255/255), marker='s', label=r'45RFSOI $\tau_c$', linewidth=2, markersize=10)

plt.yscale('log')

# Create a second y-axis object that shares the same x-axis

# ax2.set_ylim(0.01, 7)  # Set the limit for right y-axis

# Title and layout
# plt.title('Impact of Temperature on the Tau of RTN')
fig.tight_layout()  # To ensure nothing is clipped

# Show plot with a legend
ax1.legend(loc='best', bbox_to_anchor=(1, 1))  # ['45SPCLO $\tau_e$','45SPCLO $\tau_c$','45RFSOI $\tau_e$','45RFSOI $\tau_c$']
# handles = [line1, line2, line3, line4],

# Set box and grid properties
plt.legend(prop={'weight': 'normal'})  # Set legend text weight to 'normal' (non-bold)
plt.box(True)  # Show box around plot
plt.rcParams['axes.linewidth'] = 2  # Set box linewidth to 2
plt.rcParams['savefig.dpi']=300
plt.grid(True, linewidth=2, linestyle='--', color='black', alpha=0.25)  # Set grid properties

plt.show()



In [ ]:
## Added by Bozhi - Test plot in Time Domain
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import pandas as pd

df1 = df_denoise50K_7
df1.columns = ["Time", "Id_original", "Id"]
data_plot = df1.iloc[650000*4+2000:650000*(4+1) - 2000]


# Creating subplots
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True,horizontal_spacing=0.02)

# Line Plot on the left
fig.add_trace(
    go.Scatter(x=data_plot['Time'], y=data_plot['Id'], mode='lines', marker_color='blue', line=dict(color='blue')),
    row=1, col=1
)

# KDE plot on the right
# For KDE, you might need to calculate the density manually or use an external library as Plotly does not have direct KDE support
# Here's an example using a histogram for simplicity
fig.add_trace(
    go.Histogram(y=data_plot['Id'], histnorm='probability density', orientation='h', marker_color='lightblue'),
    row=1, col=2
)


# Update layout
fig.update_layout(
    font=dict(family='times new roman', size=20, color='black'),
    xaxis1=dict(title='Time [s]', linecolor='black', linewidth=2),
    yaxis1=dict(title='ID [A]', linecolor='black', linewidth=2),
    xaxis2=dict(title='Count', linecolor='black', linewidth=2),
    title="Time domain plot for Vds=0.3V",
    # margin={"l":0.1, "t":0.4},
    width=900,
    height=400,
    showlegend=False,
    template='plotly_white',
)

# Set gridline style and line width
fig.update_xaxes(showgrid=True, gridwidth=2, gridcolor='gray', griddash='dot')
fig.update_yaxes(showgrid=True, gridwidth=2, gridcolor='gray', griddash='dot')

# Display the plot
fig.show()

# Vg0.7_Vd(0.1, 0.2, 0.3, 0.4, 0.5, 0.6)